In [ ]:
!pip install bertopic[flair,gensim,spacy,use]

In [ ]:
!pip install bertopic[vision]

In [ ]:
from bertopic import BERTopic
import openai
from bertopic.representation import OpenAI
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import csv
import re
import nltk
import string

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

In [ ]:

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

In [ ]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('portuguese'))
    stop_words.update(["nº","cep","telefone","rua","avenida","endereço","fax","fones"])
    stop_words.update(["egrégia","egrégio","eg","e.g."])
    stop_words.update(["copy","reg","trade","ldquo","rdquo","lsquo","rsquo","bull","middot","sdot","ndash","mdash","cent","pound","euro","ne","frac12","frac14","frac34","deg","larr","rarr","uarr","darr","egrave","eacute","ccedil","hellip"])
    tokens = nltk.word_tokenize(text, language='portuguese')
    tokens_cleaned = [token for token in tokens if token not in stop_words]
    text_cleaned = ' '.join(tokens_cleaned)
    return text_cleaned

In [ ]:
def clean_text(doc):
    final_doc = ""
    doc = doc.lower()
    #Tenta identificar parte relevante do documento
    match = re.search(r' cabimento[^\n]*',doc)
    #Palavras irrelevantes
    match_pattern = [r'\b_+(?:\d+|[a-zA-Z]+)?\b',r'https?://\S+',r'www\.\S+',r'\S+@\S+',r'^\d{3}.\d{3}.\d{3}-\d{2}$',r'^\d{2}\.\d{3}\.\d{3}\/\d{4}\-\d{2}$',r'\d{2}/\d{2}/\d{4}[ ,]',r'procuradoria regional (federal|da união) da \d+[ªa] região',r'tribunal regional federal[ da] \d+[ªa] região',r'advocacia[ -]geral da união',r'(excelentíssimo|senhor|vice-presidente|desembargador|\(a\))',r'procuradoria[ -]geral federal',r'escritório de advocacia',r'[ superior] tribunal de justiça',r'supremo tribunal federal',r'fones',r'fax']
    subs = [''] * len(match_pattern)
    if match:
        start = match.start()
        final_doc = doc[start:]
    else:
        final_doc = doc
    for match_pattern, subs in zip(match_pattern,subs):
        final_doc = re.sub(match_pattern,subs,final_doc)
    final_doc = remove_stopwords(final_doc)
        
    return final_doc
    

In [ ]:
def remove_stopwords_tfidf(text,stop_words):
    tokens = nltk.word_tokenize(text, language='portuguese')
    tokens_cleaned = [token for token in tokens if token not in stop_words]
    text_cleaned = ' '.join(tokens_cleaned)
    return text_cleaned

## **Lê recursos a partir do dataset e faz limpeza inicial dos textos**

In [ ]:
#Lê recursos especiais
resp = pd.read_csv('REsp_completo.csv')

docs = []
num_cadastrado = []
indice = []
clean = True

for i, linha in resp.iterrows():
  tipo = type(linha['recurso'])
  try:
    #tratamento necessário, textos estavam sendo identificados como float
    if (tipo == str):
      if clean:
          doc_cleaned = clean_text(linha['recurso'])
          num_cadastrado.append(int(linha['num_tema_cadastrado']))
          docs.append(doc_cleaned)
          indice.append(i)     
      else:
          num_cadastrado.append(int(linha['num_tema_cadastrado']))
          docs.append(linha['recurso'])
          indice.append(i)
      
  except Exception as erro:
    print(f"Erro ao capturar numero de tema cadastrado {i}")
    continue

## **Cria matriz tf-idf e a partir de um limite estabelecido, cria um conjunto de stopwords e faz nova limpeza dos textos antes de inicar a modelagem de topicos**

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(docs)

vocab = vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vocab)

# limite para o valor de TF-IDF
limite_tfidf = 0.006

# Encontra as palavras com TF-IDF inferior ao limite em todos os registros
inferior_tfidf_words_all = df_tfidf.columns[df_tfidf.lt(limite_tfidf).all()]

stopwords_list = inferior_tfidf_words_all.tolist()

for i in range(len(docs)):
    docs[i]=remove_stopwords_tfidf(docs[i],stopwords_list)

In [ ]:
#print(docs[0])

In [ ]:
#print(stopwords_list)

In [ ]:
#Cria lista de temas apartir de arquivo
temas_repetitivos_eproc = pd.read_csv('temas_repetitivos.csv', sep=',' )
temas = temas_repetitivos_eproc[['tema','num_tema_cadastrado']].copy()
temas.columns = ['texto','numTema']
list_temas = list(temas.itertuples(index=False, name=None))

In [ ]:
temas_seed_list = []
temas_seed = temas_repetitivos_eproc[['tema']].copy()
temas_seed.columns = ['texto']
for indice,linha in temas_seed.iterrows():
    seed = clean_text(linha[0])
    seed = remove_punctuation(seed)
    temas_seed_list.append(seed.split())



In [ ]:
#print(temas_seed_list)

In [ ]:

len(docs)
#print(docs[470])

In [ ]:
len(num_cadastrado)

In [ ]:
#Carrega modelo pré-treinado pra criar embeddings dos textos 
#Modelo do bertopic não estava gerando corretamente topicos em portugues estava excluindo letras acentuadas e cedilha
sentence_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
topic_model = BERTopic(embedding_model=sentence_model,top_n_words=13, seed_topic_list=temas_seed_list)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_info=topic_model.get_topic_info()

In [ ]:
representacao = topic_model.get_document_info(docs)

In [ ]:
print(representacao)


In [ ]:
#features = topic_model.vectorizer_model.get_feature_names()

In [ ]:
#print(features)

In [ ]:
fig = topic_model.visualize_term_rank(log_scale=True)
fig.write_html("grafico_topicos.html")

In [ ]:
bertopic_unsupervised = pd.DataFrame()

In [ ]:
bertopic_unsupervised["indice"]=indice
bertopic_unsupervised["num_tema_cadastrado"]=num_cadastrado
bertopic_unsupervised["recurso"]=docs
bertopic_unsupervised["topicos"]=representacao['Top_n_words']

In [ ]:
print(bertopic_unsupervised)

In [ ]:
#Salva em arquivo os textos com topicos principais extraidos e numeros de temas reais cadastrados
file = 'bertopic_nao_supervisionado.csv'
bertopic_unsupervised.to_csv(file,index=False)

In [ ]:
def read_text(dataframe, linha, coluna):
    texto = dataframe.at[linha, coluna]
    return texto

In [ ]:
def sort_list(lista):
    return(sorted(lista, key = lambda x: x[1],reverse=True))

In [ ]:

def calc_similarity(topics, temas ,k,tema_real):
  lista_similaridade = []
  lista_tema_real = []
  
  for indice, tupla_num_tema in enumerate(temas):
      query_embedding = sentence_model.encode(topics)
      #print(f"Tema {indice} : {tupla_num_tema[0]}")
      tema_cleaned = clean_text(tupla_num_tema[0])
      text_embedding = sentence_model.encode(tema_cleaned)
      tensor_similaridade = util.cos_sim(query_embedding, text_embedding)
      valor_similaridade = tensor_similaridade.item()
      tupla = (tupla_num_tema[1],valor_similaridade)
      lista_similaridade.append(tupla)

  sorted_list = sort_list(lista_similaridade)
  for i, linha in enumerate(sorted_list):
      if(linha[0]==tema_real):
        lista_tema_real.append(i+1)#identifica posição do tema real no ranking
        lista_tema_real.append(linha[1]) 
        break
  ranking = sorted_list[:k]
  return ranking,lista_tema_real

In [ ]:
def create_columns(k):
  #k refere-se ao numero de elementos no ranking
  colunas = []
  colunas.append("indice")
  colunas.append("num_tema_cadastrado")
  for i in range(1, k + 1):
    nome = f"sugerido_{i}"
    colunas.append(nome)
    nome = f"similaridade_{i}"
    colunas.append(nome)
  colunas.append("posicao_tema_real")
  colunas.append("similaridade_tema_real")
  return colunas


In [ ]:
#Cria dataframe pra armazenar dados sobre textos classificados
nomes_colunas = create_columns(6)
resp_classificados = pd.DataFrame(columns=nomes_colunas)
#Cria arquivo pra armazenar resultados
#resp_classificados.to_csv('resp_classif_bertopic_unsuperv.csv', index=False)
resp_classificados.to_csv('resp_bertopic_unsuperv_filter.csv', index=False)


In [ ]:
#passa topicos de cada documento e lista de temas
for indice, linha in bertopic_unsupervised.iterrows():
  dados = []
  dados.append(indice)
  #numero do tema cadastrado por um analista
  try:
      dados.append(int(linha['num_tema_cadastrado']))
  except Exception as erro:
      print(f"Erro ao capturar numero de tema cadastrado {indice}")
      continue
  
  try:
    #retira hifen entre as palavras dos topicos
      topicos = linha['topicos'].replace("-","")    
      ranking , lista_tema_real = calc_similarity(topicos, list_temas, 6, linha['num_tema_cadastrado'])
  except Exception as erro:
      print(f"Erro calculo similaridade indice {indice}") 
      continue
  for i, tupla_num_tema in enumerate(ranking):
    #captura numero do tema sugerido e valor da similaridade
    dados.append(tupla_num_tema[0])
    dados.append(tupla_num_tema[1])
  
  if(lista_tema_real):
    dados.append(lista_tema_real[0])
    dados.append(lista_tema_real[1])
  else:
    dados.append("NA")
    dados.append("NA")
  #with open('resp_classif_bertopic_unsuperv.csv', mode='a', newline='') as arquivo:
  with open('resp_bertopic_unsuperv_filter.csv', mode='a', newline='') as arquivo:
    writer = csv.writer(arquivo)
    writer.writerow(dados)

In [ ]:
#Salva modelo
topic_model.save("is_notes/my_model_bertopic.pt", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)
